In [6]:
import pdfplumber
import pandas as pd

C:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [7]:
# Function to search for protocol ID and site num
def search_keywords(lst):
    keywords = ['Protocol ID:', 'Site Number:']
    result = []
    for item in lst:
        if all(keyword in item for keyword in keywords):
            result.append(item)
    return result

In [8]:
# Creates path to all PTA files (hard-coded, but need to automate into a script)
# PTA 1-4 are all sites from the same protocol ID / study
# PTA 5 has question split into two pages so need to write if/else statements for
pta_file_path=[
    "C:/Users/SUE03/OneDrive - Pfizer/Desktop/PTA_Assignement/PTA1.pdf",
    "C:/Users/SUE03/OneDrive - Pfizer/Desktop/PTA_Assignement/PTA2.pdf",
    "C:/Users/SUE03/OneDrive - Pfizer/Desktop/PTA_Assignement/PTA3.pdf",
    "C:/Users/SUE03/OneDrive - Pfizer/Desktop/PTA_Assignement/PTA4.pdf"
]

In [9]:
# Creates initial pandas data frame
pta_site_performance_df=pd.DataFrame()

In [10]:
# Looks through all the designated PTA files
for file_path in pta_file_path:
    
    pdf = pdfplumber.open(file_path)
    
    # Extract the Protocol Id and Site Number
    text_list=pdf.pages[0].extract_text_simple().split("\n")
    metadata=search_keywords(text_list) # Search for the element that contains the protocol id and site number
    metadata_list=metadata[0].split()
    protocol_id=metadata_list[2]
    site_number=metadata_list[-1]
    
    # Look through all of the tables, only get the tables with title matching "SITE PERFORMANCE AND RECRUITMENT"
    site_performance_tables=[]
    for num in range(len(pdf.pages)):
        page=pdf.pages[num]
        tables=page.extract_tables() # get all of the tables within that page
        for table_num in range(len(tables)): # get only the site perforance tables 
            if tables[table_num][0][0] == "SITE PERFORMANCE AND RECRUITMENT":
                site_performance_tables.append(tables[table_num][1:]) # this gets rid of "site recruitment..." table title

    # Comebine all site performance tables into one            
    site_performance_table=[]
    for i in range(len(site_performance_tables)):
        table=site_performance_tables[i]
        for j in range(len(table)):
            row=table[j]
            question=row[0]
            answer=row[-1]
            site_performance_table.append([question, answer])
        
    df=pd.DataFrame(site_performance_table)
    df=df.T
    df.columns=df.iloc[0]
    df=df.tail(-1)
    df.insert(loc=0, column='Protocol ID', value=[protocol_id])
    df.insert(loc=1, column='Site Number', value=[site_number])
    pta_site_performance_df=pta_site_performance_df.append(df, ignore_index=True)
    pta_site_performance_df=pta_site_performance_df.drop(columns=pta_site_performance_df.columns[-1])
    pdf.close()

In [11]:
pta_columns=pta_site_performance_df.columns.tolist()[2:]

In [12]:
len(pta_columns)

23

In [13]:
# Creating a default dictionary
column_mapping = {
    pta_columns[0]: 'Q1',
    pta_columns[1]: 'Q2',
    pta_columns[2]: 'Q3',
    pta_columns[3]: 'Q4',
    pta_columns[4]: 'Q5',
    pta_columns[5]: 'Q6',
    pta_columns[6]: 'Q7',
    pta_columns[7]: 'Q8',
    pta_columns[8]: 'Q9',
    pta_columns[9]: 'Q10',
    pta_columns[10]: 'Q11',
    pta_columns[11]: 'Q12',
    pta_columns[12]: 'Q13',
    pta_columns[13]: 'Q14',
    pta_columns[14]: 'Q15',
    pta_columns[15]: 'Q16',
    pta_columns[16]: 'Q17',
    pta_columns[17]: 'Q18',
    pta_columns[18]: 'Q19',
    pta_columns[19]: 'Q20',
    pta_columns[20]: 'Q21',
    pta_columns[21]: 'Q22',
    pta_columns[22]: 'Q23'
}

In [233]:
column_mapping

{'Does the Investigator have access to suitable subjects that qualify based on study target\npopulation?': 'Q1',
 'Enter the estimated number of potential subjects:': 'Q2',
 'Enter the number of clinical trials conducted by the principal Investigator that compete for\nthe subject population:': 'Q3',
 'Will the site require subject-facing materials in multiple languages?': 'Q4',
 'If Yes, specify the languages needed for Informed Consent Translation:': 'Q5',
 'Can the site complete required regulatory documents and negotiate their study budget and\ncontract in parallel?': 'Q6',
 'If No, please describe the order for completing regulatory documents and contract\nnegotiation': 'Q7',
 'What is the median cycle time from receipt of Investigator Initiation Package documents to\nsite initiation?': 'Q8',
 'How many clinical trials are being conducted by the Investigator and held at the site at the\ntime of this assessment?': 'Q9',
 'How many are Pfizer clinical trials?': 'Q10',
 'How many are 

In [14]:
pta_site_performance_df = pta_site_performance_df.rename(columns=column_mapping)

In [15]:
pta_site_performance_df

,Protocol ID,Site Number,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,Q14,Q15,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23
0,C4601003,1008,Yes,150,0,No,N/A,Yes,N/A,1-2 Weeks,...,Greater than 100,0,None of the Above,N/A,Yes,Yes,Yes,Yes,PI indicated they can enroll 150 -\n200 patien...,None
1,C4601003,1030,Yes,250,0,No,N/A,Yes,N/A,1-2 Weeks,...,Greater than 100,0,None of the Above,N/A,Yes,Yes,Yes,Yes,PI informed they can enroll\nbetween 250-300 s...,CRA discussed the C4601003 Phase 3 Protocol Sy...
2,C4601003,1079,Yes,500,0,No,N/A,Yes,N/A,8-12 weeks,...,Greater than 100,0,None of the Above,N/A,Yes,Yes,Yes,Yes,PI would like to bring on additional\nSIs for ...,None
3,C4601003,1044,Yes,250,0,No,N/A,Yes,N/A,2 weeks,...,0,0,None of the Above,N/A,Yes,Yes,Yes,Yes,Synexus marketing department is\ninvolved and ...,
